In [8]:
!pip install fake_useragent


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
de_4_1_aws_access_key_id=
de_4_1_aws_secret_access_key=

In [30]:
from fake_useragent import UserAgent
import time
import requests

def get_id(keyword):
    confirmid = {}
    ua = UserAgent()
    user_agent = ua.random
    url = "https://search.map.kakao.com/mapsearch/map.daum"
    headers = {
        'Accept': '*/*',
        'Referer': 'https://map.kakao.com/',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': f'{user_agent}'
    }
    for i in range(1, 40):
        start_time = time.time()
        params = {
            'q': keyword,
            'msFlag': 'A',
            'sort': '0',
            'page': f'{i}'
        }
        try:
            time.sleep(1)
            response = requests.get(url, headers=headers, params=params, timeout=10)  # Fix headers
            response.raise_for_status()
        except requests.exceptions.HTTPError as errh:
            print("HTTP Error:", errh)
        except requests.exceptions.ConnectionError as errc:
            print("Error Connecting:", errc)
        except requests.exceptions.Timeout as errt:
            print("Timeout Error:", errt)
        except requests.exceptions.RequestException as err:
            print("Error:", err)
        else:
            data = response.json()
            confirmid_dict = {}
            for place in data["place"]:
                confirmid_dict[place.get('confirmid')] = [place.get('lon'), place.get('lat')]
            if len(confirmid_dict) == 0:
                print(f"{i} page has no data")
                break
            else:
                confirmid.update(confirmid_dict)
    return confirmid


In [4]:
import json
import logging
import boto3

def upload_to_s3(lists, key, bucket_name, aws_access_key_id=None, aws_secret_access_key=None, aws_session_token=None):
    # 리스트를 JSON 형식으로 변환
    json_content = json.dumps(lists, ensure_ascii=False, indent=4)

    # AWS 자격 증명 설정
    aws_credentials = {
        'aws_access_key_id': aws_access_key_id,
        'aws_secret_access_key': aws_secret_access_key,
        'aws_session_token': aws_session_token,
    }

    # Boto3 S3 클라이언트 생성
    s3_client = boto3.client('s3', **aws_credentials)

    # JSON 형식의 내용을 S3로 업로드
    s3_client.put_object(Body=json_content, Bucket=bucket_name, Key=key)

    print(f"upload to {key}")


In [31]:
from datetime import datetime
from collections import OrderedDict
def get_stores(keyword, stores):

    current_time = datetime.now()
    # YYYYMMDD 형식으로 변환
    now_date = current_time.strftime("%Y%m%d")
    for store in list(stores.keys()):
        response = requests.get('https://place.map.kakao.com/main/v/' + store)
        time.sleep(1)
        data = response.json()
        
        lists = []
        hrs = []
        store_info = OrderedDict()

        basic_info = data.get('basicInfo', {})
        store_info['last_update'] = basic_info.get('source', '').get('date', '')
        store_info['name'] = basic_info.get('placenamefull', '')
        store_info['english_name'] = basic_info.get('englishname', '')
        store_info['address'] = basic_info.get('address', {}).get('newaddr', {}).get('newaddrfull', '')
        store_info['postal_code'] = basic_info.get('address', {}).get('newaddr', {}).get('bsizonno', '')
        store_info['xpos'] = basic_info.get('wpointx')    #추가
        store_info['ypos'] = basic_info.get('wpointy')    #추가
        store_info['phone'] = basic_info.get('phonenum', '')

        for period in basic_info.get('openHour', {}).get('periodList', [{}])[0].get('timeList', [{}]):
            try:
                hrs.append(period['dayOfWeek'] + ' ' + period['timeSE'])
            except KeyError as e:
                print(f"KeyError: {e} - 시간 정보 없음")
                continue
        store_info['open_hours'] = hrs
        
        store_info['homepage'] = basic_info.get('homepage', '')
        store_info['introduction'] = basic_info.get('introduction', '')
        store_info['category'] = basic_info.get('category', {}).get('catename', '')
        store_info['tags'] = basic_info.get('tags', '')

        facility_info = basic_info.get('facilityInfo', {})
        store_info['facilities'] = {
            'disabled_accessibility': facility_info.get('fordisabled', ''),
            'parking': facility_info.get('parking', ''),
            'pet_friendly': facility_info.get('pet', ''),
            'wifi': facility_info.get('wifi', '')
        }


        menu_info = data.get('menuInfo', {})
        menu_list = menu_info.get('menuList', [])
        menus = []

        for menu in menu_list:
            menu_dict = {}
            try:
                menu_dict['menu'] = menu['menu']
                menu_dict['price'] = menu.get('price', '')
                menu_dict['description'] = menu.get('desc', '')
                menu_dict['img'] = menu.get('img', '')
            except KeyError as e:
                print(f"KeyError: {e} - 메뉴 정보 불완전")
                continue
            menus.append(menu_dict)

        store_info['menus'] = menus
        lists.append(store_info.copy())  # 복사본 추가
        upload_to_s3(lists, f'kakao/{keyword}/{now_date}/{store}_info.json', 'de-4-1-bucket', de_4_1_aws_access_key_id, de_4_1_aws_secret_access_key)
        time.sleep(1)

        ##Visotors
        visitors_lists=[]
        hrs = []
        store_info2 = OrderedDict()
        s2graph_info = data.get('s2graph', {})
        if s2graph_info:
            store_info2['current_time'] = now_date
            days_of_week = ['sunday', 'monday', 'tuesday',
                        'wednesday', 'thursday', 'friday', 'saturday']
            store_info2['days'] = OrderedDict(
                (day, s2graph_info.get('day', {}).get(day, 0)) for day in days_of_week)
            store_info2['gender'] = {
                'data': s2graph_info.get('gender', {}).get('data', []),
                'labels': s2graph_info.get('gender', {}).get('labels', [])
            }
            store_info2['age'] = {
            'data': s2graph_info.get('age', {}).get('data', []),
            'labels': s2graph_info.get('age', {}).get('labels', [])
            }
            visitors_lists.append(store_info2.copy())  # 복사본 추가
            upload_to_s3(visitors_lists, f'kakao/{keyword}/{now_date}/{store}_visitor.json', 'de-4-1-bucket', de_4_1_aws_access_key_id, de_4_1_aws_secret_access_key)
            time.sleep(1)
        else:
            print("방문자 정보 없음")
        
        ## reviews
        store_info3 = OrderedDict()
        reviews_lists = []
        comment_info = data.get('comment', {})
        if comment_info:
            store_info3["comment_num"] = comment_info.get("kamapComntcnt", 0)
            store_info3["comment_sum"] = comment_info.get("scoresum", 0)
            store_info3["comment_cnt"] = comment_info.get("scorecnt", 0)
            store_info3["strengthCounts"] = comment_info.get(
                "strengthCounts", {})
            store_info3["comment_list"] = []

            while comment_info.get('hasNext') and "list" in comment_info.keys():
                last_idx = comment_info.get('list')[-1]['commentid']
                store_info3["comment_list"] += comment_info.get('list')
                response = requests.get(
                    'https://place.map.kakao.com/commentlist/v/' + store + '/' + last_idx)
                data2 = response.json()
                comment_info = data2.get('comment', {})
            if "list" in comment_info.keys():
                store_info3["comment_list"] += comment_info.get('list')
            reviews_lists.append(store_info3.copy())
            upload_to_s3(reviews_lists, f'kakao/{keyword}/{now_date}/{store}_reviews.json', 'de-4-1-bucket', de_4_1_aws_access_key_id, de_4_1_aws_secret_access_key)
            time.sleep(1)
        else:
            print("리뷰정보없음")

        ## blogs
        blogs_lists = []
        store_info4 = OrderedDict()

        blog_info = data.get('blogReview', {})
        if blog_info:
            store_info4["blog_num"] = blog_info["blogrvwcnt"]
            store_info4["blog_list"] = []
            while "moreId" in blog_info.keys():
                lastidx = blog_info["moreId"]
                store_info4["blog_list"]+=blog_info.get('list')
                response = requests.get(
                   'https://place.map.kakao.com/blogrvwlist/v/' + store + '/' + str(lastidx))
                data3 = response.json()
                blog_info = data3.get('blogReview', {})
            store_info4["blog_list"]+=blog_info.get('list')
            blogs_lists.append(store_info4.copy())
            upload_to_s3(blogs_lists, f'kakao/{keyword}/{now_date}/{store}_blogs.json', 'de-4-1-bucket', de_4_1_aws_access_key_id, de_4_1_aws_secret_access_key)
        else:
            print("블로그 리뷰 정보 없음")
    time.sleep(1)

    

In [ ]:
if __name__=="__main__":
    import time
    keywords = [
        ["경복궁", "광화문", "덕수궁", "보신각", "암사동", "창덕궁", "종묘", "가산디지털단지역", "강남역", "건대입구역"],
        ["고덕역", "고속터미널역", "교대역", "구로디지털단지역", "구로역", "군자역", "남구로역", "대림역", "동대문역", "뚝섬역"],
        ["미아사거리역", "발산역", "북한산우이역", "사당역", "삼각지역", "서울대입구역", "서울식물원역", "마곡나루역", "서울역", "선릉역"],
        ["성신여대입구역", "수유역", "신논현역", "논현역", "신도림역", "신림역", "신촌역", "이대역", "양재역", "역삼역"],
        ["연신내역", "오목교역", "목동운동장역", "왕십리역", "용산역", "이태원역", "장지역", "장한평역", "천호역", "총신대입구(이수)역"],
        ["충정로역", "합정역", "혜화역", "홍대입구역", "회기역", "-4·19거리-", "가락시장", "가로수길", "광장(전통)시장", "김포공항"],
        ["낙산공원", "이화마을", "노량진", "덕수궁길", "정동길", "방배역", "북촌한옥마을", "서촌", "성수카페거리", "수유리"],
        ["쌍문동", "압구정로데오거리", "여의도", "연남동", "영등포", "외대앞", "용리단길", "이태원", "인사동", "익선동"],
        ["창동", "청담동", "청량리", "해방촌", "경리단길", "DDP", "DMC", "강서한강공원", "고척돔", "광나루한강공원"],
        ["광화문광장", "국립중앙박물관", "용산가족공원", "난지한강공원", "남산공원", "노들섬", "뚝섬한강공원", "망원한강공원", "반포한강공원", "북서울꿈의숲"],
        ["불광천", "서리풀공원", "몽마르뜨공원", "서울숲공원", "시청광장", "아차산", "양화한강공원", "어린이대공원", "여의도한강공원", "월드컵공원"],
        ["응봉산", "이촌한강공원", "잠실종합운동장", "잠실한강공원", "잠원한강공원", "청계산", "청와대"]
    ]

    for keyword in keywords[3][2:3]:
        start_time = time.time()  # 검색 시작 시간 기록
        print(f"Starting search for {keyword} 맛집")
        stores = get_id(keyword + ' 맛집')
        dic = get_stores(keyword, stores)
        print(f"Finished search for {keyword} 맛집")
        print(f"Time taken for {keyword} 맛집 search: {time.time() - start_time} seconds")
        print("-" * 50)
